In [1]:
import numpy as np
import os
import h5py
import dask.array as da
import dask.bag as db
from collections import Counter
import re
from functools import reduce
import operator as op
import json

C:\pyhton\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Считать файлы из архива all_k в dask bag

In [4]:
path = 'D:/all_k/'

In [5]:
b = db.read_text(os.path.join(path , '*.txt'), encoding = 'ansi')

In [6]:
#кол - во партиций = кол -во файлов по умолчанию
b.npartitions

665

In [7]:
b.take(10)

('      Джилли КУПЕР\n',
 '      \n',
 '      \n',
 '      ПРУДЕНС\n',
 '      \n',
 '      \n',
 '      \n',
 '      От автора\n',
 '      Замысел "ПРУДЕНС" впервые появился у меня в 1967 году. Я написала длинный рассказ под названием "КАРТОЧНЫЙ ДОМИК", позже опубликованный в "19". Спустя время я вернулась к этому рассказу и полностью переписала его, в результате появилась "ПРУДЕНС".\n',
 '      \n')

Подсчитать кол-во русских букв

In [8]:
def count_letters(text):
    text = text.lower()
    txts = re.findall("[а-яА-ЯёЁ]", text)
    return Counter(txts)

In [9]:
b.map(count_letters).fold(op.add).compute()

Counter({'а': 21322436,
         'б': 4860996,
         'в': 11312096,
         'г': 4737949,
         'д': 8105547,
         'е': 21969368,
         'ж': 2845293,
         'з': 4495933,
         'и': 17062720,
         'й': 2911202,
         'к': 8622932,
         'л': 13464478,
         'м': 8455309,
         'н': 17572093,
         'о': 29120773,
         'п': 7335858,
         'р': 11631526,
         'с': 13965456,
         'т': 15853252,
         'у': 7384808,
         'ф': 453087,
         'х': 2189394,
         'ц': 881932,
         'ч': 3853340,
         'ш': 2129603,
         'щ': 898742,
         'ъ': 61751,
         'ы': 4848844,
         'ь': 5177834,
         'э': 1132553,
         'ю': 1510726,
         'я': 5438075,
         'ё': 447183})

Посчитать кол-во русских слов

In [10]:
rus_symb = set(chr(symb) for symb in range(ord('а'), ord('я')+1))
def is_cyrillic(word):
    if len(word) > 0:
        return bool((word[0] in rus_symb) * is_cyrillic(word[1:]))
    else: return True

In [11]:
def count_words(text):
    reslist = re.findall('\w+', text.lower())
    return Counter(filter(is_cyrillic, reslist))

In [14]:
b_1 = db.read_text('D:/all_k/БРАВО.txt', encoding = 'ansi')

In [15]:
%time
res_1 = b_1.map(count_words).fold(op.add).compute()
print(len(res_1))

Wall time: 0 ns
21335


In [16]:
res_1

Counter({'браво': 173,
         'или': 189,
         'в': 2507,
         'венеции': 217,
         'джеймс': 1,
         'фенимор': 2,
         'купер': 37,
         'предисловие': 1,
         'к': 718,
         'роману': 2,
         'первый': 13,
         'из': 499,
         'трех': 43,
         'романов': 3,
         'купера': 21,
         'на': 1448,
         'темы': 4,
         'истории': 5,
         'европейских': 5,
         'стран': 6,
         'два': 25,
         'других': 52,
         'гейденмауэр': 1,
         'и': 3986,
         'палач': 4,
         'эти': 79,
         'произведения': 2,
         'явились': 5,
         'результатом': 2,
         'пребывания': 2,
         'писателя': 5,
         'европе': 6,
         'побывал': 2,
         'англии': 1,
         'франции': 4,
         'швейцарии': 1,
         'германии': 1,
         'италии': 18,
         'свои': 123,
         'впечатления': 2,
         'от': 425,
         'посещения': 2,
         'этих': 62,
         'он': 898